Marcin Wardyński  
wtorek, 9:45

## Laboratorium 7
### 7.2 DBN

In [4]:
import importlib
import lab7_utils as utils
importlib.reload(utils)

seed = 42

In [37]:
from sklearn.base import TransformerMixin, BaseEstimator

class FrozenRBM(TransformerMixin, BaseEstimator):
    def __init__(self, rbm):
        self.rbm = rbm

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return self.rbm.transform(X)

In [48]:
from sklearn.base import clone
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

def wrap_rbm_snaps(rbms_snaps):
    rbm_tuples = []
    for i in range(len(rbms_snaps)):
        rbm_tuples.append((f"rbm_L{i+1}", rbms_snaps[i]))

    return rbm_tuples


def create_dbn(hidden_sizes, rbm_base, log_reg_base, X_train, y_train):
    rbm_snaps = []
    log_regs = []

    log_reg = clone(log_reg_base)
    pipeline = Pipeline([("log_reg", log_reg)])
    pipeline.fit(X_train, y_train)
    log_regs.append(log_reg)


    for hidden_size in hidden_sizes:
        log_reg = clone(log_reg_base)
        rbm = clone(rbm_base)
        rbm.n_components = hidden_size


        pipeline_def = []
        pipeline_def.extend(wrap_rbm_snaps(rbm_snaps))
        pipeline_def.append((f"rbm_L{len(rbm_snaps)+1}", rbm))
        pipeline_def.append(("log_reg", log_reg))
       
        pipeline = Pipeline(pipeline_def)
        pipeline.fit(X_train, y_train)

        rbm_snaps.append(FrozenRBM(rbm))
        log_regs.append(log_reg)
    
    return rbm_snaps, log_regs


In [49]:
from sklearn.metrics import classification_report

def test_dbn(rbms, log_regs, X_test, y_test):
    for i in range(len(rbms)+1):
        pipeline_def = []
        pipeline_def.extend(wrap_rbm_snaps(rbms[:i]))
        pipeline_def.append(("log_reg", log_regs[i]))

        pipeline = Pipeline(pipeline_def)
        y_pred = pipeline.predict(X_test)

        print(pipeline.named_steps.keys())
        print(classification_report(y_test, y_pred))


In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import BernoulliRBM

In [ ]:
X_train, X_test, y_train, y_test = utils.get_dataset(utils.Dataset_Select.MNIST.value, 10)

In [51]:

rbm_base = BernoulliRBM(learning_rate=0.01, n_iter=10, random_state=seed)
log_reg_base = LogisticRegression(max_iter=1000, solver='lbfgs')

hidden_sizes = [256, 128, 64]

rbms, log_regs = create_dbn(hidden_sizes, rbm_base, log_reg_base, X_train, y_train)



In [52]:
log_regs

[LogisticRegression(max_iter=1000),
 LogisticRegression(max_iter=1000),
 LogisticRegression(max_iter=1000),
 LogisticRegression(max_iter=1000)]

In [53]:
test_dbn(rbms, log_regs, X_test, y_test)

dict_keys(['log_reg'])
              precision    recall  f1-score   support

           0       0.94      0.97      0.96       980
           1       0.96      0.98      0.97      1135
           2       0.92      0.89      0.90      1032
           3       0.90      0.91      0.90      1010
           4       0.93      0.92      0.92       982
           5       0.89      0.86      0.87       892
           6       0.93      0.94      0.94       958
           7       0.93      0.91      0.92      1028
           8       0.87      0.87      0.87       974
           9       0.90      0.90      0.90      1009

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000

dict_keys(['rbm_L1', 'log_reg'])
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       980
           1       0.98      0.99      0.99      1135
           2       0.9